### Importing Libraries

In [1]:
# Uncomment to install
# !pip -q install pandas numpy requests bs4 threading

In [2]:
import pandas as pd
import threading, multiprocessing
import requests
from bs4 import BeautifulSoup

### Grand Prix Data Frame

In [3]:
grand_prix_df = pd.DataFrame({
    "round_num": [1, 2, 3, 4],
    "grand_prix": ["Bahrain", "Saudi Arabia", "Australia", "Japan"],
    "laps": [57, 50, 58, 53]
})

grand_prix_df.set_index("round_num", inplace=True)
grand_prix_df.head()

,grand_prix,laps
round_num,,
1,Bahrain,57
2,Saudi Arabia,50
3,Australia,58
4,Japan,53
